# Building an Agent to Play Atari games using Deep Q Network

First we import all the necessary libraries </font> 


In [1]:
import numpy as np
import gym
import tensorflow as tf
from tensorflow.contrib.layers import flatten, conv2d, fully_connected
from collections import deque, Counter
import random
from datetime import datetime
import time
import pickle

Now we define a function called preprocess_observation for preprocessing our input game screen. We reduce the image size
and convert the image into greyscale.

In [2]:
color = np.array([210, 164, 74]).mean()

def preprocess_observation(obs):

    # Crop and resize the image
    img = obs[20:196:2, ::2]
    
    # Convert the image to greyscale
    img = img.mean(axis=2)
    
    # Improve image contrast
    img[img==color] = 0
    
    # Next we normalize the image from -1 to +1
    img = img/255
    
    img = img.reshape(88,80,1)
    
    return img

 Let us initialize our gym environment

In [3]:
env = gym.make("SpaceInvaders-v0")
n_outputs = env.action_space.n

Okay, Now we define a function called q_network for building our Q network. We input the game state
to the Q network and get the Q values for all the actions in that state. <br><br>
We build Q network with three convolutional layers with same padding followed by a fully connected layer. 

In [4]:
tf.reset_default_graph()

def q_network(X, name_scope):
    
    # Initialize layers
    initializer = tf.contrib.layers.variance_scaling_initializer()

    with tf.variable_scope(name_scope) as scope: 

        # initialize the convolutional layers
        layer_1 = conv2d(X, num_outputs=32, kernel_size=(8,8), stride=4, padding='SAME', weights_initializer=initializer) 
        tf.summary.histogram('layer_1',layer_1)
        
        layer_2 = conv2d(layer_1, num_outputs=64, kernel_size=(4,4), stride=2, padding='SAME', weights_initializer=initializer)
        tf.summary.histogram('layer_2',layer_2)
        
        layer_3 = conv2d(layer_2, num_outputs=64, kernel_size=(3,3), stride=1, padding='SAME', weights_initializer=initializer)
        tf.summary.histogram('layer_3',layer_3)
        
        # Flatten the result of layer_3 before feeding to the fully connected layer
        flat = flatten(layer_3)

        fc = fully_connected(flat, num_outputs=128, weights_initializer=initializer)
        tf.summary.histogram('fc',fc)
        
        output = fully_connected(fc, num_outputs=n_outputs, activation_fn=None, weights_initializer=initializer)
        print(output.name)
        tf.summary.histogram('output',output)
        

        # Vars will store the parameters of the network such as weights
        vars = {v.name[len(scope.name):]: v for v in tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)} 
        return vars, output

Next we define a function called epsilon_greedy for performing epsilon greedy policy. In epsilon greedy policy we either select the best action with probability 1 - epsilon or a random action with
probability epsilon.

We use decaying epsilon greedy policy where value of epsilon will be decaying over time as we don't want to explore
forever. So over time our policy will be exploiting only good actions.

In [5]:
epsilon = 0.5
eps_min = 0.05
eps_max = 1.0
eps_decay_steps = 10800000

def epsilon_greedy(action, step):
    p = np.random.random(1).squeeze()
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        return action

Now, we initialize our experience replay buffer of length 20000 which holds the experience.

We store all the agent's experience i.e (state, action, rewards) in the experience replay buffer
and  we sample from this minibatch of experience for training the network.

In [6]:
buffer_len = 20000
exp_buffer = deque(maxlen=buffer_len)

Next, we define a function called sample_memories for sampling experiences from the memory. Batch size is the number of experience sampled
from the memory.


In [7]:
def sample_memories(batch_size):
    perm_batch = np.random.permutation(len(exp_buffer))[:batch_size]
    mem = np.array(exp_buffer)[perm_batch]
    return mem[:,0], mem[:,1], mem[:,2], mem[:,3], mem[:,4]

Now we define our network hyperparameters,

In [8]:
num_episodes = 15000
batch_size = 64
input_shape = (None, 88, 80, 1)
learning_rate = 0.00025
X_shape = (None, 88, 80, 1)
discount_factor = 0.9

global_step = 0
copy_steps = 100
steps_train = 4
start_steps = 2000

 Now let us build our primary and target Q network

In [9]:
logdir = 'logs'

# Now we define the placeholder for our input i.e game state
X = tf.placeholder(tf.float32, shape=X_shape, name="X")

# we define a boolean called in_training_model to toggle the training
in_training_mode = tf.placeholder(tf.bool, name="in_training_mode")

# we build our Q network, which takes the input X and generates Q values for all the actions in the state
mainQ, mainQ_outputs = q_network(X, 'mainQ')

# similarly we build our target Q network
targetQ, targetQ_outputs = q_network(X, 'targetQ')

# define the placeholder for our action values
X_action = tf.placeholder(tf.int32, shape=(None,))
Q_action = tf.reduce_sum(targetQ_outputs * tf.one_hot(X_action, n_outputs), axis=-1, keep_dims=True)

# Copy the primary Q network parameters to the target Q network
copy_op = [tf.assign(main_name, targetQ[var_name]) for var_name, main_name in mainQ.items()]
copy_target_to_main = tf.group(*copy_op)

# Compute and optimize loss using gradient descent optimizer
# define a placeholder for our output i.e action
y = tf.placeholder(tf.float32, shape=(None,1))

# now we calculate the loss which is the difference between actual value and predicted value
loss = tf.reduce_mean(tf.square(y - Q_action))

# we use adam optimizer for minimizing the loss
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

loss_summary = tf.summary.scalar('LOSS', loss)
merge_summary = tf.summary.merge_all()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
mainQ/fully_connected_1/BiasAdd:0
targetQ/fully_connected_1/BiasAdd:0
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


 Now we start the tensorflow session and run the model,

In [10]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    init.run()
    rewards = []
    # for each episode
    for i in range(num_episodes):
        done = False
        obs = env.reset()
        epoch = 0
        episodic_reward = 0
        actions_counter = Counter() 
        episodic_loss = []

        # while the state is not the terminal state
        while not done:

#             env.render()
        
            # get the preprocessed game screen
            obs = preprocess_observation(obs)

            # feed the game screen and get the Q values for each action
            actions = mainQ_outputs.eval(feed_dict={X:[obs], in_training_mode:False})

            # get the action
            action = np.argmax(actions, axis=-1)
            actions_counter[str(action)] += 1 

            # select the action using epsilon greedy policy
            action = epsilon_greedy(action, global_step)
            
            # now perform the action and move to the next state, next_obs, receive reward
            next_obs, reward, done, _ = env.step(action)

            # Store this transistion as an experience in the replay buffer
            exp_buffer.append([obs, action, preprocess_observation(next_obs), reward, done])
            
            # After certain steps, we train our Q network with samples from the experience replay buffer
            if global_step % steps_train == 0 and global_step > start_steps:
                
                # sample experience
                o_obs, o_act, o_next_obs, o_rew, o_done = sample_memories(batch_size)

                # states
                o_obs = [x for x in o_obs]

                # next states
                o_next_obs = [x for x in o_next_obs]

                # next actions
                next_act = mainQ_outputs.eval(feed_dict={X:o_next_obs, in_training_mode:False})


                # reward
                y_batch = o_rew + discount_factor * np.max(next_act, axis=-1) * (1-o_done) 

                # merge all summaries and write to the file
                mrg_summary = merge_summary.eval(feed_dict={X:o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act, in_training_mode:False})
                file_writer.add_summary(mrg_summary, global_step)

                # now we train the network and calculate loss
                train_loss, _ = sess.run([loss, training_op], feed_dict={X:o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act, in_training_mode:True})
                episodic_loss.append(train_loss)
            
            # after some interval we copy our main Q network weights to target Q network
            if (global_step+1) % copy_steps == 0 and global_step > start_steps:
                copy_target_to_main.run()
                
            obs = next_obs
            epoch += 1
            global_step += 1
            episodic_reward += reward
        print('Episode', i,'Epoch', epoch, 'Reward', episodic_reward, 'Global Step', global_step)
        rewards.append(episodic_reward)
        if i % 50 == 0:
            saver.save(sess, './models/trained-model-'+ str(i))
            pickle.dump(rewards, open('./rewards/trained-model-'+str(i)+'.pck', 'wb+'))
    saver.save(sess, './trained-model')
    pickle.dump(rewards, open('./rewards.pck', 'wb+'))
    env.close()
    
    

Episode 0 Epoch 578 Reward 110.0 Global Step 578
Episode 1 Epoch 656 Reward 210.0 Global Step 1234
Episode 2 Epoch 493 Reward 40.0 Global Step 1727
Episode 3 Epoch 667 Reward 100.0 Global Step 2394
Episode 4 Epoch 428 Reward 65.0 Global Step 2822
Episode 5 Epoch 644 Reward 155.0 Global Step 3466
Episode 6 Epoch 401 Reward 30.0 Global Step 3867
Episode 7 Epoch 654 Reward 80.0 Global Step 4521
Episode 8 Epoch 615 Reward 105.0 Global Step 5136
Episode 9 Epoch 793 Reward 125.0 Global Step 5929
Episode 10 Epoch 573 Reward 105.0 Global Step 6502
Episode 11 Epoch 828 Reward 185.0 Global Step 7330
Episode 12 Epoch 571 Reward 90.0 Global Step 7901
Episode 13 Epoch 534 Reward 80.0 Global Step 8435
Episode 14 Epoch 490 Reward 55.0 Global Step 8925
Episode 15 Epoch 514 Reward 75.0 Global Step 9439
Episode 16 Epoch 396 Reward 140.0 Global Step 9835
Episode 17 Epoch 518 Reward 55.0 Global Step 10353
Episode 18 Epoch 627 Reward 145.0 Global Step 10980
Episode 19 Epoch 671 Reward 105.0 Global Step 116

Episode 158 Epoch 761 Reward 125.0 Global Step 109920
Episode 159 Epoch 657 Reward 105.0 Global Step 110577
Episode 160 Epoch 590 Reward 110.0 Global Step 111167
Episode 161 Epoch 761 Reward 240.0 Global Step 111928
Episode 162 Epoch 659 Reward 135.0 Global Step 112587
Episode 163 Epoch 389 Reward 80.0 Global Step 112976
Episode 164 Epoch 766 Reward 155.0 Global Step 113742
Episode 165 Epoch 799 Reward 210.0 Global Step 114541
Episode 166 Epoch 658 Reward 110.0 Global Step 115199
Episode 167 Epoch 390 Reward 45.0 Global Step 115589
Episode 168 Epoch 654 Reward 75.0 Global Step 116243
Episode 169 Epoch 862 Reward 210.0 Global Step 117105
Episode 170 Epoch 469 Reward 110.0 Global Step 117574
Episode 171 Epoch 859 Reward 120.0 Global Step 118433
Episode 172 Epoch 1360 Reward 395.0 Global Step 119793
Episode 173 Epoch 812 Reward 130.0 Global Step 120605
Episode 174 Epoch 894 Reward 120.0 Global Step 121499
Episode 175 Epoch 1171 Reward 180.0 Global Step 122670
Episode 176 Epoch 1209 Reward

Episode 305 Epoch 1065 Reward 335.0 Global Step 217375
Episode 306 Epoch 496 Reward 135.0 Global Step 217871
Episode 307 Epoch 402 Reward 110.0 Global Step 218273
Episode 308 Epoch 759 Reward 155.0 Global Step 219032
Episode 309 Epoch 760 Reward 145.0 Global Step 219792
Episode 310 Epoch 1191 Reward 515.0 Global Step 220983
Episode 311 Epoch 829 Reward 410.0 Global Step 221812
Episode 312 Epoch 756 Reward 180.0 Global Step 222568
Episode 313 Epoch 468 Reward 40.0 Global Step 223036
Episode 314 Epoch 1043 Reward 345.0 Global Step 224079
Episode 315 Epoch 560 Reward 80.0 Global Step 224639
Episode 316 Epoch 747 Reward 335.0 Global Step 225386
Episode 317 Epoch 633 Reward 135.0 Global Step 226019
Episode 318 Epoch 701 Reward 80.0 Global Step 226720
Episode 319 Epoch 804 Reward 210.0 Global Step 227524
Episode 320 Epoch 579 Reward 120.0 Global Step 228103
Episode 321 Epoch 805 Reward 170.0 Global Step 228908
Episode 322 Epoch 478 Reward 30.0 Global Step 229386
Episode 323 Epoch 390 Reward 

Episode 458 Epoch 1203 Reward 335.0 Global Step 322142
Episode 459 Epoch 794 Reward 180.0 Global Step 322936
Episode 460 Epoch 811 Reward 210.0 Global Step 323747
Episode 461 Epoch 652 Reward 180.0 Global Step 324399
Episode 462 Epoch 635 Reward 75.0 Global Step 325034
Episode 463 Epoch 487 Reward 105.0 Global Step 325521
Episode 464 Epoch 1012 Reward 490.0 Global Step 326533
Episode 465 Epoch 735 Reward 120.0 Global Step 327268
Episode 466 Epoch 790 Reward 210.0 Global Step 328058
Episode 467 Epoch 635 Reward 110.0 Global Step 328693
Episode 468 Epoch 523 Reward 60.0 Global Step 329216
Episode 469 Epoch 690 Reward 225.0 Global Step 329906
Episode 470 Epoch 615 Reward 110.0 Global Step 330521
Episode 471 Epoch 751 Reward 340.0 Global Step 331272
Episode 472 Epoch 502 Reward 110.0 Global Step 331774
Episode 473 Epoch 535 Reward 105.0 Global Step 332309
Episode 474 Epoch 599 Reward 55.0 Global Step 332908
Episode 475 Epoch 737 Reward 210.0 Global Step 333645
Episode 476 Epoch 638 Reward 

Episode 611 Epoch 677 Reward 120.0 Global Step 428633
Episode 612 Epoch 918 Reward 310.0 Global Step 429551
Episode 613 Epoch 482 Reward 80.0 Global Step 430033
Episode 614 Epoch 777 Reward 165.0 Global Step 430810
Episode 615 Epoch 592 Reward 120.0 Global Step 431402
Episode 616 Epoch 488 Reward 130.0 Global Step 431890
Episode 617 Epoch 660 Reward 130.0 Global Step 432550
Episode 618 Epoch 407 Reward 15.0 Global Step 432957
Episode 619 Epoch 711 Reward 105.0 Global Step 433668
Episode 620 Epoch 525 Reward 105.0 Global Step 434193
Episode 621 Epoch 366 Reward 50.0 Global Step 434559
Episode 622 Epoch 389 Reward 110.0 Global Step 434948
Episode 623 Epoch 730 Reward 180.0 Global Step 435678
Episode 624 Epoch 778 Reward 160.0 Global Step 436456
Episode 625 Epoch 493 Reward 120.0 Global Step 436949
Episode 626 Epoch 689 Reward 105.0 Global Step 437638
Episode 627 Epoch 604 Reward 155.0 Global Step 438242
Episode 628 Epoch 705 Reward 380.0 Global Step 438947
Episode 629 Epoch 440 Reward 25

...

Episode 14332 Epoch 996 Reward 295.0 Global Step 11260557
Episode 14333 Epoch 864 Reward 230.0 Global Step 11261421
Episode 14334 Epoch 1133 Reward 510.0 Global Step 11262554
Episode 14335 Epoch 712 Reward 265.0 Global Step 11263266
Episode 14336 Epoch 703 Reward 280.0 Global Step 11263969
Episode 14337 Epoch 575 Reward 185.0 Global Step 11264544
Episode 14338 Epoch 634 Reward 235.0 Global Step 11265178
Episode 14339 Epoch 1581 Reward 370.0 Global Step 11266759
Episode 14340 Epoch 1213 Reward 530.0 Global Step 11267972
Episode 14341 Epoch 1056 Reward 295.0 Global Step 11269028
Episode 14342 Epoch 581 Reward 215.0 Global Step 11269609
Episode 14343 Epoch 605 Reward 235.0 Global Step 11270214
Episode 14344 Epoch 601 Reward 195.0 Global Step 11270815
Episode 14345 Epoch 664 Reward 280.0 Global Step 11271479
Episode 14346 Epoch 696 Reward 285.0 Global Step 11272175
Episode 14347 Epoch 1532 Reward 800.0 Global Step 11273707
Episode 14348 Epoch 1104 Reward 210.0 Global Step 11274811
Episode 

Episode 14473 Epoch 1084 Reward 190.0 Global Step 11380752
Episode 14474 Epoch 640 Reward 320.0 Global Step 11381392
Episode 14475 Epoch 622 Reward 235.0 Global Step 11382014
Episode 14476 Epoch 677 Reward 245.0 Global Step 11382691
Episode 14477 Epoch 600 Reward 265.0 Global Step 11383291
Episode 14478 Epoch 693 Reward 305.0 Global Step 11383984
Episode 14479 Epoch 725 Reward 295.0 Global Step 11384709
Episode 14480 Epoch 1148 Reward 250.0 Global Step 11385857
Episode 14481 Epoch 709 Reward 285.0 Global Step 11386566
Episode 14482 Epoch 1096 Reward 185.0 Global Step 11387662
Episode 14483 Epoch 627 Reward 140.0 Global Step 11388289
Episode 14484 Epoch 1050 Reward 510.0 Global Step 11389339
Episode 14485 Epoch 642 Reward 235.0 Global Step 11389981
Episode 14486 Epoch 652 Reward 245.0 Global Step 11390633
Episode 14487 Epoch 1111 Reward 210.0 Global Step 11391744
Episode 14488 Epoch 725 Reward 280.0 Global Step 11392469
Episode 14489 Epoch 716 Reward 265.0 Global Step 11393185
Episode 1

Episode 14614 Epoch 1198 Reward 255.0 Global Step 11500824
Episode 14615 Epoch 620 Reward 225.0 Global Step 11501444
Episode 14616 Epoch 738 Reward 290.0 Global Step 11502182
Episode 14617 Epoch 605 Reward 235.0 Global Step 11502787
Episode 14618 Epoch 676 Reward 250.0 Global Step 11503463
Episode 14619 Epoch 675 Reward 180.0 Global Step 11504138
Episode 14620 Epoch 620 Reward 260.0 Global Step 11504758
Episode 14621 Epoch 1063 Reward 320.0 Global Step 11505821
Episode 14622 Epoch 622 Reward 235.0 Global Step 11506443
Episode 14623 Epoch 642 Reward 155.0 Global Step 11507085
Episode 14624 Epoch 1130 Reward 225.0 Global Step 11508215
Episode 14625 Epoch 716 Reward 270.0 Global Step 11508931
Episode 14626 Epoch 513 Reward 120.0 Global Step 11509444
Episode 14627 Epoch 1108 Reward 200.0 Global Step 11510552
Episode 14628 Epoch 817 Reward 225.0 Global Step 11511369
Episode 14629 Epoch 1141 Reward 210.0 Global Step 11512510
Episode 14630 Epoch 1147 Reward 425.0 Global Step 11513657
Episode 

Episode 14755 Epoch 1009 Reward 470.0 Global Step 11617758
Episode 14756 Epoch 861 Reward 205.0 Global Step 11618619
Episode 14757 Epoch 613 Reward 265.0 Global Step 11619232
Episode 14758 Epoch 1256 Reward 295.0 Global Step 11620488
Episode 14759 Epoch 704 Reward 285.0 Global Step 11621192
Episode 14760 Epoch 660 Reward 280.0 Global Step 11621852
Episode 14761 Epoch 1037 Reward 330.0 Global Step 11622889
Episode 14762 Epoch 717 Reward 280.0 Global Step 11623606
Episode 14763 Epoch 635 Reward 125.0 Global Step 11624241
Episode 14764 Epoch 694 Reward 280.0 Global Step 11624935
Episode 14765 Epoch 721 Reward 280.0 Global Step 11625656
Episode 14766 Epoch 669 Reward 290.0 Global Step 11626325
Episode 14767 Epoch 1190 Reward 235.0 Global Step 11627515
Episode 14768 Epoch 1096 Reward 375.0 Global Step 11628611
Episode 14769 Epoch 1203 Reward 220.0 Global Step 11629814
Episode 14770 Epoch 717 Reward 300.0 Global Step 11630531
Episode 14771 Epoch 661 Reward 175.0 Global Step 11631192
Episode 

Episode 14896 Epoch 740 Reward 280.0 Global Step 11732004
Episode 14897 Epoch 1130 Reward 515.0 Global Step 11733134
Episode 14898 Epoch 733 Reward 245.0 Global Step 11733867
Episode 14899 Epoch 614 Reward 225.0 Global Step 11734481
Episode 14900 Epoch 1524 Reward 800.0 Global Step 11736005
Episode 14901 Epoch 693 Reward 350.0 Global Step 11736698
Episode 14902 Epoch 1091 Reward 185.0 Global Step 11737789
Episode 14903 Epoch 1135 Reward 210.0 Global Step 11738924
Episode 14904 Epoch 717 Reward 290.0 Global Step 11739641
Episode 14905 Epoch 591 Reward 185.0 Global Step 11740232
Episode 14906 Epoch 746 Reward 280.0 Global Step 11740978
Episode 14907 Epoch 1632 Reward 510.0 Global Step 11742610
Episode 14908 Epoch 585 Reward 145.0 Global Step 11743195
Episode 14909 Epoch 643 Reward 235.0 Global Step 11743838
Episode 14910 Epoch 707 Reward 305.0 Global Step 11744545
Episode 14911 Epoch 1119 Reward 230.0 Global Step 11745664
Episode 14912 Epoch 802 Reward 435.0 Global Step 11746466
Episode 